# Demo File
This demo tests the functionality of the EchoStateNetwork with different time series

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from ESN import EchoStateNetwork, MackeyGlassGenerator,GinfActivator, visualize_reservoir, BandPassNetwork

ginf_activator = GinfActivator(V_min=-2, V_max=2, resolution=200, offset=True)


def mse(test, predictions):
    return np.mean((predictions - test) ** 2)

## Steady State Conductance Activator

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5.4, 5.4), layout='constrained')
x = np.arange(100)
for nn, ax in enumerate(axs):
    ax.plot(x, x)
    if nn == 1:
        ax.set_title('Manual ticks')
        ax.set_yticks(np.arange(0, 100.1, 100/3))
        xticks = np.arange(0.50, 101, 20)
        xlabels = [f'\\${x:1.2f}' for x in xticks]
        ax.set_xticks(xticks, labels=xlabels)
    else:
        ax.set_title('Automatic ticks')

In [ ]:
lookup_table = ginf_activator.get_lookup_table()

plt.figure(figsize=(3, 2))
plt.plot(
    lookup_table[0],
    lookup_table[1],
    label=r"$g_{\infty}(V)$",
    color="blue",
    linewidth=2,
)
plt.xlabel(r"$V \;[\text{V}]$", fontsize=14)
plt.ylabel(r"$g_{\infty}(V)$", fontsize=14)
#plt.grid(True, linestyle="--", alpha=0.6)
plt.xticks(np.arange(-2,2.5,1),fontsize=12)
plt.yticks(np.arange(-1,1.05,0.5),fontsize=12)
#plt.legend(fontsize=12)        
plt.savefig("output/ginf_activator_plot.pdf",bbox_inches='tight')
plt.savefig("output/ginf_activator_plot.png",bbox_inches='tight')

## ESN with Mackey-Glass time series

In [ ]:
esn = EchoStateNetwork(
    input_dim=1,
    reservoir_size=10,
    output_dim=1,
    leaking_rate=1,
    step_size=0.1,
    time_scale=1,
    spectral_radius=0.9, #guarantee_ESP could change spectral radius if True
    sparsity=0.85,
    input_scaling=0.5,
    regularization=1e-6,
    weight_seed=35,
    activation=ginf_activator.activate,
    guarantee_ESP=True,
)

In [ ]:
# Generate Mackey-Glass time series
dt_mg = 1
mg_series = MackeyGlassGenerator(tau=17, n=100000, n_samples=50000, dt=dt_mg)

# Normalize data between -1 and 1
mg_series = 2 * (mg_series - mg_series.min()) / (mg_series.max() - mg_series.min()) - 1

# Create input/output pairs for time series prediction
inputs = mg_series[:-1].reshape(-1, 1)
targets = mg_series[1:].reshape(-1, 1)

# Split into train and test sets
train_len = 40000
test_len = 4000
train_inputs = inputs[:train_len]
train_targets = targets[:train_len]
test_inputs = inputs[train_len : train_len + test_len]
test_targets = targets[train_len : train_len + test_len]

In [ ]:
esn = EchoStateNetwork(
    input_dim=1,
    reservoir_size=100,
    output_dim=1,
    leaking_rate=0.9,
    step_size=dt_mg,
    time_scale=1/0.45,
    spectral_radius=0.9,
    sparsity=0.85,
    input_scaling=0.5,
    regularization=1e-6,
    washout = round(train_len/400),
    activation=ginf_activator.activate,
    guarantee_ESP=False,
)

# Train on first part of series
esn.fit(train_inputs, train_targets)

# Generate predictions on test set
teacher_ratio = 0.2
predictions = esn.predict(test_inputs, teacher_ratio=teacher_ratio)

# Calculate prediction error after network settled from np.zeroes initialization
print(f"{mse(test_targets, predictions) = }")

print(f"{esn.physical_length = }")
teacher_mode = int(len(test_targets)*teacher_ratio)
print(f"{teacher_mode = }")

In [ ]:
bpn = BandPassNetwork(
    input_dim=1,
    reservoir_size=100,
    output_dim=1,
    leaking_rate=0.9,
    step_size=dt_mg,
    time_scale_avg=1/0.45,
    time_scale_std=0.4,
    spectral_radius=0.9,
    sparsity=0.85,
    input_scaling=0.5,
    regularization=1e-6,
    washout = round(train_len/400),
    activation=ginf_activator.activate,
    guarantee_ESP=False,
)

# Train on first part of series
bpn.fit(train_inputs, train_targets)

# Generate predictions on test set
teacher_ratio = 0.2
predictions = bpn.predict(test_inputs, teacher_ratio=teacher_ratio)

# Calculate prediction error after network settled from np.zeroes initialization
print(f"{mse(test_targets, predictions) = }")

print(f"{esn.physical_length = }")
teacher_mode = int(len(test_targets)*teacher_ratio)
print(f"{teacher_mode = }")

In [ ]:
cutoff_min = 500
cutoff_max = 1500

plt.figure(figsize=(6, 3))
plt.axvline(x=teacher_mode, linestyle="--", color='black')
plt.text(cutoff_min+180, 1.2, 'Teacher-forced:\nOnly next step predicted', dict(size=10), horizontalalignment='center')
plt.text(cutoff_max-180, 1.2, 'Free-running:\nNetwork output is next input', dict(size=10), horizontalalignment='center')
plt.plot(test_targets, label="True MG series", alpha=1, color='blue')
plt.plot(predictions, label="ESN predictions", linestyle="--", alpha=1, color='red')
# plt.title("Mackey-Glass Time Series Prediction (First 400 Steps)")
plt.xlabel("Time [s]", fontsize=14)
plt.ylabel("Amplitude [a.u.]", fontsize=14)
plt.xlim(cutoff_min, cutoff_max)
plt.ylim(-1.15, 1.15)
plt.legend(loc='lower right',framealpha=1)
plt.savefig("output/mg_prediction_plot.pdf")
plt.savefig("output/mg_prediction_plot.png")

In [ ]:
# Plot phase space reconstruction
plt.figure(figsize=(8, 6))
plt.plot(
    test_targets[:-1][:5000],
    test_targets[1:][:5000],
    "b-",
    alpha=0.5,
    label="True",
)
plt.plot(
    predictions[:-1][:5000],
    predictions[1:][:5000],
    "r--",
    alpha=0.5,
    label="Predicted",
)
plt.title("Phase Space Reconstruction")
plt.xlabel("x(t)")
plt.ylabel("x(t+1)")
plt.legend()
plt.savefig("output/mg_phase_space_plot.pdf")
plt.savefig("output/mg_phase_space_plot.png")

In [ ]:
#visualize_reservoir(esn)

## ESN with Harmonic Wave time series

In [ ]:
# Generate synthetic data (simple sine wave prediction)
t = np.linspace(0, 80 * np.pi, 800)
dt_harmonic = 80 * np.pi/800
data = np.sin(t) * np.cos(1.2*t)

# Create input/output pairs for time series prediction
inputs = data[:-1].reshape(-1, 1)
targets = data[1:].reshape(-1, 1)

In [ ]:
# Initialize and train ESN
esn = EchoStateNetwork(
    input_dim=1,
    reservoir_size=15,
    output_dim=1,
    leaking_rate=0.9,
    step_size=dt_harmonic,
    time_scale=2,
    spectral_radius=0.8,
    sparsity=0.75,
    input_scaling=0.5,
    regularization=1e-6,
    washout=0,
    activation=ginf_activator.activate,
)

# Train on synthetic data
esn.fit(inputs, targets)

# Generate predictions
teacher_ratio=0.25
predictions = esn.predict(inputs, teacher_ratio=teacher_ratio)

# Calculate prediction error after network settled from np.zeroes initialization
print(f"{mse(targets, predictions) = }")

teacher_mode = int(len(targets)*teacher_ratio)
print(f"{teacher_mode = }")

In [ ]:
# Initialize and train ESN
bpn = BandPassNetwork(
    input_dim=1,
    reservoir_size=15,
    output_dim=1,
    leaking_rate=0.9,
    step_size=dt_harmonic,
    time_scale_avg=4,
    time_scale_std=3,
    spectral_radius=0.8,
    sparsity=0.75,
    input_scaling=0.5,
    regularization=1e-6,
    washout=0,
    activation=ginf_activator.activate,
    weight_seed = 1,
)

# Train on synthetic data
bpn.fit(inputs, targets)

# Generate predictions
teacher_ratio=0.25
bpn_predictions = bpn.predict(inputs, teacher_ratio=teacher_ratio)

# Calculate prediction error after network settled from np.zeroes initialization
print(f"{mse(targets, bpn_predictions) = }")

teacher_mode = int(len(targets)*teacher_ratio)
print(f"{teacher_mode = }")

In [ ]:
cutoff_min = 100
cutoff_max = 440

plt.figure(figsize=(6, 3))
plt.axvline(x=teacher_mode, linestyle="--", color='black')
plt.text(cutoff_min+50, 1.2, 'Teacher-forced:\nOnly next step predicted', dict(size=10), horizontalalignment='center')
plt.text(cutoff_max-100, 1.2, 'Free-running:\nNetwork output is next input', dict(size=10), horizontalalignment='center')
plt.plot(targets, label="True signal",color='blue',linewidth=2)
plt.plot(predictions, label="ESN prediction", linestyle="--",color='red',linewidth=2)
plt.plot(bpn_predictions, label="BPN prediction", linestyle="--",color='lightgreen',linewidth=2)
plt.xlabel("Time [s]", fontsize=14)
plt.ylabel("Amplitude [a.u.]", fontsize=14)
plt.xlim(cutoff_min, cutoff_max)
plt.ylim(-1.15, 1.15)
plt.legend(loc='lower right',framealpha=1)
plt.savefig("output/sine_prediction_plot.pdf")
plt.savefig("output/sine_prediction_plot.png")

In [ ]:
visualize_reservoir(esn)

## ESN with Ventilator Pressure Prediction data

In [ ]:
import polars as pl

In [ ]:
data = pl.read_csv(r"data\ventilator-pressure-prediction\train.csv")[:100000]

data = data.to_dummies(columns=["R", "C"])

split = int(len(data) * 0.8)

train_x = data.drop('id','breath_id',"pressure")[:split].to_numpy()
train_y = data[["pressure"]][:split].to_numpy()
test_x = data.drop('id','breath_id',"pressure")[split:].to_numpy()
test_y = data[["pressure"]][split:].to_numpy()

data.drop('id','breath_id',"pressure")[:split].head()

In [ ]:
esn = EchoStateNetwork(
    input_dim=train_x.shape[1],
    reservoir_size=80,
    output_dim=train_y.shape[1],
    leaking_rate=0.66,
    step_size=0.18,
    time_scale=3.1,
    spectral_radius=0.16,
    sparsity=0.75,
    input_scaling=0.1,
    regularization=0.00047855525,
    washout = 100,
    activation=ginf_activator.activate,
    guarantee_ESP=True,
    progress_bar=True,
)

esn.fit(train_x, train_y)

In [ ]:
predictions = esn.predict(test_x, teacher_ratio=1)
print(f"Test MSE: {mse(test_y, predictions)}")

In [ ]:
plt.figure(figsize=(20, 6))
plt.plot(range(len(test_y)), test_y, label="True pressure")
plt.plot(range(len(test_y)), predictions, label="ESN prediction", linestyle="--",)
plt.title("ESN Predictions vs True Pressure")
plt.xlabel("Time")
plt.ylabel("Pressure")
plt.xlim(4000, 5000)
plt.legend()
plt.show()

## PWDB

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from ESN import PulseEchoStateNetwork
import numpy as np
from sklearn.utils import shuffle
from tqdm import tqdm

In [ ]:
pathlist = Path(r'data/pulse-wave-database/PWs_csv/csv').glob('**/*.csv')
dfs = []
for path in tqdm(pathlist):
    df = pd.read_csv(path)
    _, df["filename"], df["Type"] = path.name.split("_")
    df["Type"] = df["Type"].str.replace(r"\.csv$", "", regex=True)
    dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)
data = pd.read_csv(r'data\pulse-wave-database\m.csv')
data.index = data.index + 1
df = final_df.merge(data, left_on="Subject Number", right_index=True, how="left")
pulse_columns = [col for col in df.columns if col.startswith(" pt")]
df["pulse"] = df[pulse_columns].apply(lambda row: row.dropna().tolist(), axis=1)

# Drop the original pulse columns
df = df.drop(columns=pulse_columns)
df = df[df.Type == 'P']
df = df.dropna(subset=['SI'])
df = df.reset_index(drop=True)

In [ ]:
n = randint(0, len(df))
pulse_data = df.iloc[n]["pulse"]

plt.figure(figsize=(10, 4))
plt.plot(pulse_data, label="Pulse Wave", linewidth=1)
plt.xlabel("Time (arbitrary units)")
plt.ylabel("Amplitude")
#plt.title(f"Pulse Waveform for {df.iloc[n]["filename"]} ({n})")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_merged = df[["filename", "SI", "pulse"]].copy()
#df_merged["SI"]=(df_merged["SI"]-df_merged["SI"].min())/(df_merged["SI"].max()-df_merged["SI"].min())
df_merged = shuffle(df_merged, random_state=42)

train_samples = []
test_samples = []
num_samples = 3000

unique_filenames = df_merged["filename"].unique()
for filename in unique_filenames:
    class_subset = df_merged[df_merged["filename"] == filename]

    train_samples.append(class_subset.iloc[:num_samples])
    test_samples.append(class_subset.iloc[num_samples:int(num_samples*1.2)])

train_set = pd.concat(train_samples).sample(frac=1, random_state=42)
test_set = pd.concat(test_samples).sample(frac=1, random_state=42)

train_x = list(train_set["pulse"])
train_y = train_set["SI"].values.reshape(-1, 1)
test_x = list(test_set["pulse"])
test_y = test_set["SI"].values.reshape(-1, 1)

In [ ]:
esn = PulseEchoStateNetwork(
    input_dim=1,
    reservoir_size=1000,
    output_dim=1,
    leaking_rate=0.9,
    step_size=1,
    time_scale=50,
    spectral_radius=0.4,
    sparsity=0.9,
    input_scaling=2.3758091737239763e-6,
    regularization=0.00047855525,
    washout = 3,
    guarantee_ESP=False,
    progress_bar=True,
)

esn.fit(train_x, train_y)
predictions = esn.predict(test_x)

In [ ]:
np.mean((predictions - test_y) ** 2)

One key detail is that in “pulses” mode, the code uses only the final reservoir state of each pulse to produce a single prediction. That single state often fails to capture large variations unless the reservoir is both large and driven strongly. In other words, if you have long pulses with strong peaks, reducing them to one final state makes the readout behave like an “average” of the pulse.

You can try the following adjustments:

Increase input scaling
A larger input_scaling can push reservoir states out of the mild range of tanh, leading to stronger dynamics.

Revisit guarantee_ESP
The Yildiz “guarantee” is conservative: it forces all initial reservoir weights to be nonnegative and then randomly flips signs. This dampens extremes. Try setting guarantee_ESP=False so weights have a more typical ESN distribution.

Use a bigger reservoir or smaller regularization
The final readout has to reconstruct the output from a single vector. If that vector is small or lacks variety, the predictions will be muted. A larger reservoir or weaker regularization (
𝜆
λ) can sometimes help.

Consider predicting multiple time steps
If your true target is a whole time series (with large peaks) rather than a single scalar per pulse, try the “timestep” mode. That way the readout can adapt at each time step and will not compress entire pulses into one state.

In [ ]:
predictions = np.asarray(predictions).ravel()
test_y = np.asarray(test_y).ravel()

n = 50
indices = np.arange(n)
width = 0.35

plt.figure(figsize=(16, 4))
plt.bar(indices - width/2, predictions[:n], width=width, label='Predicted SI')
plt.bar(indices + width/2, test_y[:n], width=width, label='Actual SI')
plt.title('First 50 Predictions vs. Actual SI')
plt.xlabel('Sample index')
plt.ylabel('SI value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_merged = df[["filename", "SI", "pulse"]].copy()
#df_merged["SI"]=(df_merged["SI"]-df_merged["SI"].min())/(df_merged["SI"].max()-df_merged["SI"].min())
df_merged = shuffle(df_merged, random_state=42)

train_samples = []
test_samples = []
num_samples = 100

unique_filenames = df_merged["filename"].unique()
for filename in unique_filenames:
    class_subset = df_merged[df_merged["filename"] == filename]

    train_samples.append(class_subset.iloc[:num_samples])
    test_samples.append(class_subset.iloc[num_samples:int(num_samples*1.2)])

train_set = pd.concat(train_samples).sample(frac=1, random_state=42)
test_set = pd.concat(test_samples).sample(frac=1, random_state=42)

train_x = list(train_set["pulse"])
train_y = train_set["SI"].values.reshape(-1, 1)
test_x = list(test_set["pulse"])
test_y = test_set["SI"].values.reshape(-1, 1)

In [ ]:
esn_loc = PulseEchoStateNetwork(
    input_dim=1,
    reservoir_size=1000,
    output_dim=1,
    leaking_rate=0.9,
    step_size=1,
    time_scale=75,
    spectral_radius=0.4,
    sparsity=0.9,
    input_scaling=2.3758091737239763e-6,
    regularization=0.00047855525,
    washout = 3,
    guarantee_ESP=False,
    progress_bar=True,
)

esn_loc.fit(train_x, train_y)
predictions = esn_loc.predict(test_x)